# Лабораторная работа №6
## Синтезатор частот и ФАПЧ
### Звягин М.

Импорт библиотек

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

Ниже представлена схема синтезатора частот на основе ФАПЧ. Для ее реализации каждый блок будет представлен в виде функции (блок детектора фазы, блок петлевого фильтра, блок VCO).  

В данной работе не будет рассмотрена версия фазового детектора с "зарядовым насосом".

<img src = "pll.png" align = "middle">  

Ниже представлена линеализированная схема PLL

<img src = "pll_lineal.png" align = "middle">  

Сначала рассмотрим работу PLL как отдельной схемы. После проверки ее работы сделаем на ее основе синтезотор частот

Хорошие источники:

http://www.dsplib.ru/content/pll/pll.html

http://www.dsplib.ru/content/dpll/dpll.html

## Схема ФАПЧ
ФАПЧ состоит из следующих блоков:
1) Фазовый детектор
2) Петлевой фильтр
3) Генератор, управляемый напряжением

### Фазовый детектор. 
Вычисляет разность фаз между сигналов с опорного осциллятора (входного сигнала) и сигнала с ГУН


In [ ]:
#Фазовый детектор, представленный ниже, представляет собой обычный умножитель,
#на выходе которого получаются компонента с удвоенной частотой и компонента, зависящая от разности фаз
#Работает с действительными сигналами
def Phase_Detector_Mult(input_vco_sample, input_osc_sample, K_pd):
    phase_error = input_vco_sample*input_osc_sample
    phase_detector_output = K_pd*phase_error
    return phase_detector_output, phase_error

#Фазовый детектор, представленный ниже, на выходе дает не сигнал, зависящий от разности фаз, а непосредственно разность фаз
#Работает с комплексными сигналами, предстваленными в виде комплексной экспоненты
def Phase_Detector_Ideal(input_vco_sample, input_osc_sample, K_pd):
    phase_error = np.angle(input_osc_sample*np.conjugate(input_vco_sample))
    phase_detector_output = K_pd*phase_error
    return phase_detector_output, phase_error

fs = 1e5
Ts = 1/fs
f1 = 2e3
f2 = 2.1e3
K_pd = 1

t = np.arange(0, 2000)/fs
s_1 = np.exp(1j*(2*np.pi*f1*t - np.pi/2))
s_2 = np.exp(1j*(2*np.pi*f2*t - np.pi/3))

s_res = np.zeros(shape=np.shape(s_1))
s_res_2 = np.zeros(shape=np.shape(s_1))
for i in np.arange(0, len(s_1)):
    s_res[i], _ = Phase_Detector_Mult(np.real(s_1[i]), np.real(s_2[i]), K_pd)
    s_res_2[i], _ = Phase_Detector_Ideal(s_1[i], s_2[i], K_pd)

plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(t*1e3, s_1)
plt.plot(t*1e3, s_2)
plt.title('Сигналы на входе фазового детектора')
plt.ylabel('Амплиитуда')
plt.xlabel('Время, t, мс')
plt.grid()

plt.subplot(3, 1, 2)
plt.plot(t*1e3, s_res, '-')
plt.title('Сигнал на выходе фазового детектора (умножитель)')
plt.ylabel('Амплиитуда')
plt.xlabel('Время, t, мс')
plt.grid()
plt.tight_layout()

plt.subplot(3, 1, 3)
plt.plot(t*1e3, s_res_2, '-')
plt.title('Сигнал на выходе фазового детектора (прямая разность фаз)')
plt.ylabel('Амплиитуда')
plt.xlabel('Время, t, мс')
plt.grid()
plt.tight_layout()

По полученным графикам видно, что фазовый детектор, реализованный как непосредственная разность фаз, отслеживает фазу сигнала, зависящего от разности фаз (это низкочастотная компонента сигнала, полученная с выхода фазового детектора, реализованного в виде умножителя)

Полученный график также показывает работу идеального фазового детектора, который имеет полный рабочий диапазон от -pi до pi, не подвержен шумам и не имеет мертвой зоны

Добавим в функцию фазового детектора некоторые неидеальности. Так как на таком высоком уровне нельзя внести неидеальности в умножиетель (и чтобы не моделировать каким-либо образом работу умножителя), неидеальности будут введены только в функцию фазового детектора, который реализован как непосредственная разность фаз.

В качестве неисправностей добавим небольшой случайный шум и мертвую зону

In [ ]:
def Phase_Detector_Non_Ideal(input_vco_sample, input_osc_sample, K_pd, dead_zone, jitter_std):
    phase_error = np.angle(input_osc_sample*np.conjugate(input_vco_sample))

    if abs(phase_error) < dead_zone:
        phase_error = 0.0
    elif phase_error > 0:
        phase_error -= dead_zone/2
    else:
        phase_error += dead_zone/2
        
    phase_error += np.random.normal(0, jitter_std)
    phase_detector_output = K_pd*phase_error
    
    return phase_detector_output, phase_error


In [ ]:
dead_zone = 0.25
jitter_std = 0.01

phase_error_log = np.zeros_like(s_1)
phase_detector_output_log = np.zeros_like(s_2)
for i in np.arange(0, len(s_1)):
    phase_detector_output_log[i], phase_error_log[i] = Phase_Detector_Non_Ideal(s_1[i], s_2[i], K_pd, dead_zone, jitter_std)

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(t*1e3, s_1)
plt.plot(t*1e3, s_2)
plt.title('Тестовые сигналы во временной области')
plt.xlabel('t, мс')
plt.ylabel('Амплитуда')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(phase_detector_output_log, label = 'Неидеальный фазовый детектор')
plt.plot(s_res_2, '--', label = 'Идеальный фазовый детектор')
plt.title('Выход фазового детектора')
plt.ylabel('Амплитуда')
plt.grid()
plt.legend()
plt.tight_layout()


Теперь на графиках видно появление мертвой зоны и небольшого джиттера

### Петлевой фильтр. 
Используется для преобразования выхода фазового детектора в управляющее значение напряжения для ГУН. От настроек коэффициентов петлевого фильтра будет зависеть компромисс между временем установления (захвата) и точностью в период удержания. Также от значений зависит полоса удержания и полоса пропускания системы ФАПЧ.

В данной работе в качестве петлевого фильтра был выбран пропорционально-интегрирующий фильтр

<img src = "loop_filt.png" align = "middle">  

Формула математического описания работы фильтра:
$$e(n) = K_p \cdot v(n) + K_i \cdot \sum_{i=0}^{n} v(i)$$ 

Формула передаточной функции фильтра:
$$H(s) = K_p + \frac{K_i}{s}, s = j\omega$$

In [ ]:
def Loop_Filt(current_sample, K_i, K_p):
    global integral_out
    
    proportional_out = current_sample*K_p
    integral_out += current_sample
    filtered_out = proportional_out + K_i*integral_out
    
    return integral_out, filtered_out

Рассмотрим влияние фильтра на функцию единичной ступеньки

In [ ]:
test_signal = np.heaviside(np.arange(-100, 100), 0.5)
K_i = 0.001
K_p = 0.5
integral_out = 0

integral_out_log = []
phase_error_filtered_log = []


for i in np.arange(0, len(test_signal)):
    out_1, out_2 = Loop_Filt(test_signal[i], K_i, K_p)
    integral_out_log.append(out_1)
    phase_error_filtered_log.append(out_2)
    

integral_out_log = np.reshape(integral_out_log, (len(test_signal), 1))
phase_error_filtered_log = np.reshape(phase_error_filtered_log, (len(test_signal), 1))

plt.figure()
plt.plot(np.arange(0, len(phase_error_filtered_log))/fs*1e6, phase_error_filtered_log, '-', label = 'Выход фильтра')
plt.plot(np.arange(0, len(test_signal))/fs*1e6, test_signal, '--', label = 'Вход фильтра')
plt.title('Зависимость выхода фильтра от времени')
plt.xlabel('Время t, мкс ')
plt.ylabel('Напряжение, В')
plt.grid()
plt.legend()


Рассмотрим влияние фильтра на синусоидальную функцию

In [ ]:
f = 1000
t = np.arange(0, 2*fs/f)/fs
test_signal = np.sin(2*np.pi*f*t)
K_i = 0.001
K_p = 1
integral_out = 0

integral_out_log = []
phase_error_filtered_log = []


for i in np.arange(0, len(test_signal)):
    out_1, out_2 = Loop_Filt(test_signal[i], K_i, K_p)
    integral_out_log.append(out_1)
    phase_error_filtered_log.append(out_2)
    

integral_out_log = np.reshape(integral_out_log, (len(test_signal), 1))
phase_error_filtered_log = np.reshape(phase_error_filtered_log, (len(test_signal), 1))

plt.figure()
plt.plot(np.arange(0, len(phase_error_filtered_log))/fs*1e6, phase_error_filtered_log, '-', label = 'Выход фильтра')
plt.plot(np.arange(0, len(test_signal))/fs*1e6, test_signal, '--', label = 'Вход фильтра')
plt.title('Зависимость выхода фильтра от времени')
plt.xlabel('Время t, мкс ')
plt.ylabel('Напряжение, В')
plt.grid()
plt.legend()

signal_test = phase_error_filtered_log

Рассмотрим влияние фильтра на одиночный импульс

In [ ]:
test_signal = np.zeros((500, 1))
test_signal[250] = 1

K_i = 0.5
K_p = 1
integral_out = 0

integral_out_log = []
phase_error_filtered_log = []


for i in np.arange(0, len(test_signal)):
    out_1, out_2 = Loop_Filt(test_signal[i], K_i, K_p)
    integral_out_log.append(out_1)
    phase_error_filtered_log.append(out_2)
    

integral_out_log = np.reshape(integral_out_log, (len(test_signal), 1))
phase_error_filtered_log = np.reshape(phase_error_filtered_log, (len(test_signal), 1))

plt.figure()
plt.plot(np.arange(0, len(phase_error_filtered_log))/fs*1e6, phase_error_filtered_log, '-', label = 'Выход фильтра')
plt.plot(np.arange(0, len(test_signal))/fs*1e6, test_signal, '--', label = 'Вход фильтра')
plt.title('Зависимость выхода фильтра от времени')
plt.xlabel('Время t, мкс ')
plt.ylabel('Напряжение, В')
plt.grid()
plt.legend()

Рассмотрим АЧХ петлевого фильтра

In [ ]:
w = np.logspace(-5, 5, 10000)
K_i = 0.8
K_p = 1
H = K_p + K_i / (1j * w)
magnitude = np.abs(H) 
phase = np.angle(H, deg=True)

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.semilogx(w/(2*np.pi), 20 * np.log10(magnitude))
plt.title('АЧХ петлевого фильтра')
plt.xlabel('Частота, Гц')
plt.ylabel('Уровень, дБ')
plt.grid(which='both', linestyle='--', linewidth=0.7)

plt.subplot(2, 1, 2)
plt.semilogx(w/(2*np.pi), phase)
plt.title('Фазовая характеристика PI-фильтра')
plt.xlabel('Частота, Гц')
plt.ylabel('Фаза, градусы')
plt.grid(True, which='both', linestyle='--')
plt.tight_layout()



При стремлении частоты к 0 амплитуда сигнала на выходе фильтра растет, так как будет накапливать интегратор. 
При увеличении частоты амплитуда сигнала стремится к постоянному значению, так как средний выход интегратора будет равен 0, а на выходе будет сигнал с амплитудой равной произведению амплитуды входного сигнала на пропорциональный коэффициент K_p

Построим зависимости АЧХ от коэффициентов Ki и Kp

In [ ]:
w = np.logspace(-5, 5, 10000)
K_i = np.arange(1, 10)/10
K_p = np.arange(1, 10)/10

plt.figure(figsize=(10, 6))

for k_i in K_i:
    H = 1 + k_i / (1j * w)
    magnitude = np.abs(H) 
    phase = np.angle(H, deg=True)
    
    plt.subplot(2, 1, 1)
    plt.semilogx(w/(2*np.pi), 20 * np.log10(magnitude), label = f'K_i = {k_i}, K_p = 1')
    plt.title('АЧХ петлевого фильтра')
    plt.xlabel('Частота, Гц')
    plt.ylabel('Уровень, дБ')
    plt.grid()
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.semilogx(w/(2*np.pi), phase, label = f'K_i = {k_i}, K_p = 1')
    plt.title('Фазовая характеристика PI-фильтра')
    plt.xlabel('Частота, Гц')
    plt.ylabel('Фаза, градусы')
    plt.grid(True, which='both', linestyle='--')
    plt.tight_layout()
    plt.legend()
    
    
    

### Генератор, управляемый напряжение
По сигналу ошибки с петлевого фильтра он генерирует локальный сигнал.

Математическая формула:
1) Выходной сигнал с ГУН
$$\cos \left( \omega_0 \cdot t + K_{vco} \cdot \int_{0}^{t} e(t) \, dt \right)$$

2) Мгновенная частота
$$\omega(t) = \omega_0 + K_0 \cdot e(t)$$

Где e(t) - управляющий сигнал ошибки с выхода петлевого фильтра

In [ ]:
def VCO(phase_error_filtered_sample, K_vco, f_vco, t):
    global vco_phase
    vco_phase += K_vco*phase_error_filtered_sample
    vco_freq = 2*np.pi*f_vco*t + vco_phase
    vco_output = np.exp(1j*vco_freq)
    return vco_output, vco_freq, vco_phase

In [ ]:
vco_phase = 0
vco_output = np.zeros((np.shape(signal_test)))
vco_freq = np.zeros((np.shape(signal_test)))
vco_phase_ = np.zeros((np.shape(signal_test)))


for i in np.arange(1, len(signal_test)):
    vco_output[i], vco_freq[i], vco_phase_[i] = VCO(signal_test[i], 1, 1, t[i])
    
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(np.arange(0, len(signal_test))/fs*1e6, signal_test, '-', label = 'Управляющий сигнал')
plt.title('Управляющий сигнал с выхода петлевого фильтра')
plt.xlabel('Время t, мкс ')
plt.grid()
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(np.arange(0, len(vco_freq))/fs*1e6, vco_freq, '-', label = 'Частота ГУН')
plt.title('Мгновенная частота ГУН')
plt.xlabel('Время t, мкс ')
plt.grid()
plt.legend()
plt.tight_layout()

Из полученных графиков видно, что мгновенная частота ГУН растет при положительном значении управляющего напряжения (чем выше значение, тем быстрее рост) и уменьшается при отрицательном значении, что соответствует его корректной работы 

### Система ФАПЧ

Соберем разобранные блоки к единую систему

In [ ]:
fs = 1e6
Ts = 1/fs
vco_phase = 0
integral_out = 0

f_vco = 1500
f_sig = f_vco + 500
t = np.arange(0, 100*fs/f_sig)/fs

K_vco = 1
K_pd = 1

w_resonans = 2*np.pi*500
demp_factor = 0.5

g_1 = 2*(1 - np.exp(-w_resonans*demp_factor*Ts)*np.cos(w_resonans*np.sqrt(1-demp_factor**2)*Ts))
g_2 = np.exp(-2*w_resonans*demp_factor*Ts) - 1 + g_1

K_i = g_2/(K_vco*K_pd)
K_p = g_1/(K_vco*K_pd)


signal_input = np.sin(2*np.pi*f_sig*t + np.pi/3)

phase_detector_out = np.zeros((np.shape(signal_input)))
phase_detector_out[0] = K_pd*signal_input[0]*np.cos(2*np.pi*f_vco*t[0])

loop_filt_out = np.zeros((np.shape(signal_input)))
loop_filt_out[0] = K_p*phase_detector_out[0]

phase_vco = np.zeros((np.shape(signal_input)))
freq_vco = np.zeros((np.shape(signal_input)))

vco_out = np.zeros((np.shape(signal_input)))
vco_out[0], freq_vco[0], phase_vco[0] = np.real(VCO(0, K_vco, f_vco, t[0]))

signal_error = np.zeros((np.shape(signal_input)))
signal_out = np.zeros((np.shape(signal_input)))


for i in np.arange(1, len(signal_input)):
    
    phase_detector_out[i], _ = Phase_Detector_Mult(signal_input[i], vco_out[i-1], K_pd)
    _, loop_filt_out[i] = Loop_Filt(phase_detector_out[i], K_i, K_p)
    vco_out[i], freq_vco[i], phase_vco[i] = np.real(VCO(loop_filt_out[i], K_vco, f_vco, t[i]))
    signal_out[i] = np.sin(freq_vco[i])
    signal_error[i] = signal_input[i] - signal_out[i]
    
plt.figure(figsize=(10,10))
plt.subplot(5, 1, 5)
plt.plot(np.arange(0, len(signal_error))/fs*1e6, signal_error, '-', label = 'Сигнал ошибки')
plt.title('Ошибка ФАПЧ')
plt.xlabel('Время t, мкс ')
plt.ylabel('Амплитуда')
plt.grid()
plt.legend()
 
plt.subplot(5, 1, 1)
plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
plt.title('Вызод фазового детектора')
plt.xlabel('Время t, мкс ')
plt.ylabel('Амплитуда')
plt.grid()
plt.legend()

plt.subplot(5, 1, 2)
plt.plot(np.arange(0, len(loop_filt_out))/fs*1e6, loop_filt_out, '-', label = 'Выход петлевого фильтра')
plt.title('Вызод петлевого фильтра')
plt.xlabel('Время t, мкс ')
plt.ylabel('Амплитуда')
plt.grid()
plt.legend()

plt.subplot(5, 1, 4)
plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '--', label = 'Входной сигнал')
plt.title('Сравнение входного и выходного сигналов')
plt.xlabel('Время t, мкс ')
plt.grid()
plt.legend()

plt.subplot(5, 1, 3)
plt.plot(np.arange(0, len(freq_vco))/fs*1e6, freq_vco/(2*np.pi*t), '-', label = 'Частота ГУН')
plt.title('Частота ГУН')
plt.xlabel('Время t, мкс ')
plt.grid()
plt.legend()
plt.tight_layout()


Построим АЧХ для ФАПЧ

In [ ]:
w = np.logspace(-5, 5, 10000)
w_resonans = 2*np.pi*500
demp_factor = 0.5

H = (2*demp_factor*w_resonans*(1j * w) + w_resonans**2)/((1j * w)**2 + 2*demp_factor*w_resonans*(1j * w) +  w_resonans**2)
magnitude = np.abs(H) 
phase = np.angle(H, deg=True)

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.semilogx(w/(2*np.pi), 20 * np.log10(magnitude))
plt.title('АЧХ ФАПЧ')
plt.xlabel('Частота, Гц')
plt.ylabel('Уровень, дБ')
plt.grid(which='both', linestyle='--', linewidth=0.7)

plt.subplot(2, 1, 2)
plt.semilogx(w/(2*np.pi), phase)
plt.title('Фазовая характеристика ФАПЧ')
plt.xlabel('Частота, Гц')
plt.ylabel('Фаза, градусы')
plt.grid(True, which='both', linestyle='--')
plt.tight_layout()



Для фиксированных значений резонансной частоты и коэффициента ослабления, а также коэффициентов петлевого фильтра, фазового детектора и генератора, управляемого напряжением, определим полосу захвата и полосу удержания.

Полоса захвата - максимальное отклонение входной частоты, при котором петля сможет захватить и синхронизировать сигнал ГУН

Полоса удержания - максимальное отклонение входной частоты (когда петля уже синхронизирована), при котором петля не потеряет синхронизацию


Для удобства напишем функцию, которая выполняет работу ФАПЧ

In [ ]:
def PLL(fs, f_vco, f_sig, p_sig, K_vco, K_pd, w_resonans, demp_factor, visible):
    
    Ts = 1/fs
    t = np.arange(0, 100*fs/f_sig)/fs

    g_1 = 2*(1 - np.exp(-w_resonans*demp_factor*Ts)*np.cos(w_resonans*np.sqrt(1.0-demp_factor**2)*Ts))
    g_2 = np.exp(-2*w_resonans*demp_factor*Ts) - 1 + g_1

    K_i = g_2/(K_vco*K_pd)
    K_p = g_1/(K_vco*K_pd)

    signal_input = np.sin(2*np.pi*f_sig*t + p_sig)

    phase_detector_out = np.zeros((np.shape(signal_input)))
    phase_detector_out[0] = K_pd*signal_input[0]*np.cos(2*np.pi*f_vco*t[0])

    loop_filt_out = np.zeros((np.shape(signal_input)))
    loop_filt_out[0] = K_p*phase_detector_out[0]

    phase_vco = np.zeros((np.shape(signal_input)))
    freq_vco = np.zeros((np.shape(signal_input)))

    vco_out = np.zeros((np.shape(signal_input)))
    vco_out[0], freq_vco[0], phase_vco[0] = np.real(VCO(0, K_vco, f_vco, t[0]))

    signal_error = np.zeros((np.shape(signal_input)))
    signal_out = np.zeros((np.shape(signal_input)))


    for i in np.arange(1, len(signal_input)):
        
        phase_detector_out[i], _ = Phase_Detector_Mult(signal_input[i], vco_out[i-1], K_pd)
        _, loop_filt_out[i] = Loop_Filt(phase_detector_out[i], K_i, K_p)
        vco_out[i], freq_vco[i], phase_vco[i] = np.real(VCO(loop_filt_out[i], K_vco, f_vco, t[i]))
        signal_out[i] = np.sin(freq_vco[i])
        signal_error[i] = signal_input[i] - signal_out[i]
    
    if visible == True:
        plt.figure(figsize=(12, 6))
        plt.subplot(4, 1, 1)
        plt.plot(np.arange(0, len(signal_error))/fs*1e6, signal_error, '-', label = 'Сигнал ошибки')
        plt.title(f'Ошибка ФАПЧ при исходной разнице {np.abs(f_sig - f_vco)} Гц')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 2)
        plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
        plt.title('Вызод фазового детектора')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(4, 1, 3)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '--', label = 'Входной сигнал')
        plt.title('Сравнение входного и выходного сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 4)
        plt.plot(np.arange(0, len(freq_vco))/fs*1e6, freq_vco/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        
    return signal_error, phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco

Проверка работоспособности

In [ ]:
vco_phase = 0
integral_out = 0
w_resonans = 2*np.pi*100
demp_factor = 1
_, _, _, _, _, _ = PLL(fs, 1500, 1800, np.pi/3, K_vco, K_pd, w_resonans, demp_factor, True)

In [ ]:
fs = 1e6

f_vco = 1500
f_step = 50
f_sig = np.arange(1, 30)*f_step

p_sig = 0

K_vco = 1
K_pd = 1

w_resonans = 2*np.pi*100
demp_factor = 0.5

for freq_signal in f_sig:
    vco_phase = 0
    integral_out = 0
    _, _, _, _, _, _ = PLL(fs, f_vco, freq_signal, p_sig, K_vco, K_pd, w_resonans, demp_factor, True)

По представленным результатам можно заключить, что для заданных параметров система ФАПЧ при исходной частоте ГУН 1500 Гц (то есть при изначально несинхронизированном состоянии), смогла захватить входной сигнал, когда его частота была равна 1150 Гц, то есть когда разница между частотой входного сигнала и исходной частотой ГУН составляла 350 Гц.

Для определения полосы удержания было проведено аналогичное моделирование (с модифицированной функцией, чтобы система изначально находилась в синхронизированном состоянии). Был создан сигнал путем конкатенации сигналов с частотами, которые отличаются от соседних на 50 Гц. При 20 таких сигналов система каждый раз подстраивалась и не выходила из состояния синхронизации.

Ниже будет просто приведен пример моделирования с переключением частоты


In [ ]:
def PLL_mod(fs, f_vco, K_vco, K_pd, w_resonans, demp_factor, visible):
    
    Ts = 1/fs
    t = np.arange(0, 1e6)/fs

    g_1 = 2*(1 - np.exp(-w_resonans*demp_factor*Ts)*np.cos(w_resonans*np.sqrt(1-demp_factor**2)*Ts))
    g_2 = np.exp(-2*w_resonans*demp_factor*Ts) - 1 + g_1

    K_i = g_2/(K_vco*K_pd)
    K_p = g_1/(K_vco*K_pd)

    # signal_input = (np.sin(2*np.pi*f_vco*t[0:len(t)//2]))
    signal_input = np.hstack((np.sin(2*np.pi*1700*t[0:len(t)//2]), np.sin(2*np.pi*2300*t[len(t)//2:len(t)])))
    # f_step = 250
    # for i in np.arange(1, 2):
    #     signal_input = np.hstack((signal_input, np.sin(2*np.pi*(f_vco + f_step*i)*t[(len(t)//2)*i : (len(t)//2)*(i+1)])))

    phase_detector_out = np.zeros((np.shape(signal_input)))
    phase_detector_out[0] = K_pd*signal_input[0]*np.cos(2*np.pi*f_vco*t[0])

    loop_filt_out = np.zeros((np.shape(signal_input)))
    loop_filt_out[0] = K_p*phase_detector_out[0]

    phase_vco = np.zeros((np.shape(signal_input)))
    freq_vco = np.zeros((np.shape(signal_input)))

    vco_out = np.zeros((np.shape(signal_input)))
    vco_out[0], freq_vco[0], phase_vco[0] = np.real(VCO(0, K_vco, f_vco, t[0]))

    signal_error = np.zeros((np.shape(signal_input)))
    signal_out = np.zeros((np.shape(signal_input)))


    for i in np.arange(1, len(signal_input)):
        
        phase_detector_out[i], _ = Phase_Detector_Mult(signal_input[i], vco_out[i-1], K_pd)
        _, loop_filt_out[i] = Loop_Filt(phase_detector_out[i], K_i, K_p)
        vco_out[i], freq_vco[i], phase_vco[i] = np.real(VCO(loop_filt_out[i], K_vco, f_vco, t[i]))
        signal_out[i] = np.sin(freq_vco[i])
        signal_error[i] = signal_input[i] - signal_out[i]
    
    if visible == True:
        plt.figure(figsize=(12, 6))
        plt.subplot(4, 1, 1)
        plt.plot(np.arange(0, len(signal_error))/fs*1e6, signal_error, '-', label = 'Сигнал ошибки')
        plt.title('Ошибка ФАПЧ')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 2)
        plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
        plt.title('Вызод фазового детектора')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(4, 1, 3)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '--', label = 'Входной сигнал')
        plt.title('Сравнение входного и выходного сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 4)
        plt.plot(np.arange(0, len(freq_vco))/fs*1e6, freq_vco/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        
    return signal_error, phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco, phase_vco

In [ ]:
vco_phase = 0
integral_out = 0
w_resonans = 2*np.pi*100
signal_error, phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco, phase_vco = PLL_mod(fs, 1600, K_vco, K_pd, w_resonans, demp_factor, True)

После первой подстройки была изменена частота входного сигнала и ФАПЧ снова подстроился под нее

Также было выяснено, что от значения резонансной частоты зависит скорость подстройки и ошибка ФАПЧ.

Ниже проведено моделирование при фиксированной частоте входного сигнала и изменяющей резонансной частоты.

In [ ]:
fs = 1e6

f_vco = 1500
f_sig = 2000
p_sig = 0

K_vco = 1
K_pd = 1


w_resonans = np.arange(1, 10)*2*np.pi*100
demp_factor = 0.5

for w_res in w_resonans:
    vco_phase = 0
    integral_out = 0
    _, _, _, _, _, _ = PLL(fs, f_vco, f_sig, p_sig, K_vco, K_pd, w_res, demp_factor, True)

Полученные результаты моделирования показывают, что при увеличении резонансной частоты увеличивается полоса захвата ФАПЧ, уменьшается время захвата, но но также увеличивается ошибка ФАПЧ во время удержания

## Синтезатор частот
На основе полученного ФАПЧ сделаем синтезатор частот. В данной реализации сильно упрощен момент с делителями частот, так как система настроена на гармонические сигналы, а корректный делитель нужно делать с прямоугольными импульсами и соответственно вводить в схему блоки конвертации гармонического в прямоугольный сигнал.

In [ ]:
def Freq_Synt(fs, f_osc, R, N, K_vco, K_pd, w_resonans, demp_factor, visible):
    Ts = 1/fs
    t = np.arange(0, 2000*fs/f_osc)/fs

    g_1 = 2*(1 - np.exp(-w_resonans*demp_factor*Ts)*np.cos(w_resonans*np.sqrt(1.0-demp_factor**2)*Ts))
    g_2 = np.exp(-2*w_resonans*demp_factor*Ts) - 1 + g_1

    K_i = g_2/(K_vco*K_pd)
    K_p = g_1/(K_vco*K_pd)

    signal_input = np.sin(2*np.pi*f_osc/R*t)
    f_vco = f_osc/R

    phase_detector_out = np.zeros((np.shape(signal_input)))
    phase_detector_out[0] = K_pd*signal_input[0]*np.cos(2*np.pi*f_vco*t[0])

    loop_filt_out = np.zeros((np.shape(signal_input)))
    loop_filt_out[0] = K_p*phase_detector_out[0]

    phase_vco = np.zeros((np.shape(signal_input)))
    freq_vco = np.zeros((np.shape(signal_input)))

    vco_out = np.zeros((np.shape(signal_input)))
    vco_out[0], freq_vco[0], phase_vco[0] = np.real(VCO(0, K_vco, f_vco, t[0]))

    signal_out = np.zeros((np.shape(signal_input)))


    for i in np.arange(1, len(signal_input)):
        
        phase_detector_out[i], _ = Phase_Detector_Mult(signal_input[i], vco_out[i-1], K_pd)
        _, loop_filt_out[i] = Loop_Filt(phase_detector_out[i], K_i, K_p)
        _, freq_vco[i], _ = (VCO(loop_filt_out[i], K_vco, f_vco, t[i]))
        vco_out[i] = np.cos(freq_vco[i]/N)
        signal_out[i] = np.sin(freq_vco[i])
    
    if visible == True:
        plt.figure(figsize=(12, 6))    
        plt.subplot(4, 1, 1)
        plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
        plt.title('Вызод фазового детектора')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 2)
        plt.plot(np.arange(0, len(loop_filt_out))/fs*1e6, loop_filt_out, '-', label = 'Выход петлевого фильтра')
        plt.title('Выход петлевого фильтра')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 3)
        plt.plot(np.arange(0, len(freq_vco))/fs*1e6, freq_vco/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(4, 1, 4)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал с ГУН')
        plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '-', label = 'Входной сигнал после делителя R')
        plt.title(f'Сравнение входного и выходного сигнала при R = {R} и N = {N}')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        
    return phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco

Путем моделирования было выявлено, что синтезатор частот с реализованным ФАПЧ работает достаточно плохо, так как при фиксированном значении частоты резонанса может генерировать частоты в очень ограниченном диапазоне.

Ниже приведен пример с генерацией частот с высокой кратностью (N > 10) и с низкой кратностью (N < 10)

#### N > 10

In [ ]:
w_resonans = 2*np.pi*2500
demp_factor = 0.5
fs = 1e7
f_osc = 1e5

R = 100
N = np.arange(10, 20, 1)

K_vco = 1
K_pd = 1

for n in N:
    vco_phase = 0
    integral_out = 0
    phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco = Freq_Synt(fs, f_osc, R, n, K_vco, K_pd, w_resonans, demp_factor, True)

Анализ результатов показал, что при резонансной частоте 2500 Гц синтезатор частот почти корректно генерирует частоты от 20 кГц до 40 кГц с шагом 2 кГц. Однако даже при установившемся состоянии можно заметить, что на выходе получается не чистый гармонический сигнал, а гармонический сигнал, у которого немного "ходит" частота (сигнал то шире, то уже)

Проанализируем спектр полученного сигнала для каждого случая

In [ ]:
def Plot_Spectrum(signal, fs, N):
    n = len(signal)
    yf = np.fft.fft(signal)
    amplitude = np.abs(yf) / n
    amplitude = amplitude[:n//2]
    xf = np.fft.fftfreq(n, 1/fs)[:n//2]

    plt.figure(figsize=(10, 5))
    plt.title(f'Спектр выходного сигнала при N = {N}')
    plt.xlabel('Частота, Гц')
    plt.semilogy(xf, amplitude, label='Амплитудный спектр')
    plt.xlim([0, 2e5])
    plt.grid()

In [ ]:


w_resonans = 2*np.pi*2500
demp_factor = 0.5
fs = 1e7
f_osc = 1e5

R = 100
N = np.arange(10, 20, 1)

K_vco = 1
K_pd = 1

for n in N:
    vco_phase = 0
    integral_out = 0
    phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco = Freq_Synt(fs, f_osc, R, n, K_vco, K_pd, w_resonans, demp_factor, False)
    Plot_Spectrum(signal_out, fs, n)

По полученным спектрам видно, что он достаточно широкий, то есть много побочных частот (правда, большинство этих частот получаеются в начальные моменты времени, когда ФАПЧ подстраивает свою частоту). Однако анализ спектра вблизи требуемой частоты показывает, что на выходе получается не четкий нармонический сигнал (чем выше выходная частота, тем ниже нужный частотный пик относительно остальных)

#### N < 10

In [ ]:
w_resonans = 2*np.pi*1000
demp_factor = 0.5
fs = 1e7
f_osc = 1e5

R = 100
N = np.arange(3, 8)

K_vco = 1
K_pd = 1

for n in N:
    vco_phase = 0
    integral_out = 0
    phase_detector_out, signal_out, signal_input, loop_filt_out, freq_vco = Freq_Synt(fs, f_osc, R, n, K_vco, K_pd, w_resonans, demp_factor, True)
    Plot_Spectrum(signal_out, fs, n)

По полученным результатам видно, что при значении резонансной частоты контура 1 кГц корректно были сгенерированы сигналы при N = 3, 4, 5, что в целом подтвержает их спектр. Сигналы же при N = 6, 7 являются некорреткными, так как за установленное время и при заданных параметрах система ФАПЧ не подстроилась.

Вывод:

В данной лабораторной работе была разработана и промоделирована система ФАПЧ и синтезатор частоты на ее основе.
ФАПЧ состоит из следующих блоков:

1) Фазовый детектор

В работе было реализовано 2 вида фазовых детекторов: на основе умножителя и через непосредственное вычисление фаз. Для второго варианта была рассмотрена ситуация с добавлением неидельностей (мертвая зона и джиттер). Однако дальнейшие моделирования были проведены использованием только фазового детектора на основе обычного умножителя.

2) Петлевой фильтр

Он необходим, чтобы преобразовывать сигнал с фазового детектора в управляющий сигнал для ГУН. Для данной работы в качестве петлевого фильтра был реализован пропорционально-интегрирующий фильтр. При моделировании подбор его коэффициентов осуществлялся с помощью формулы с резонансной частотой и коэффициентом ослабления в качестве входных параметров.

3) ГУН

Генератор, управляемый напряжением, генерирует гармоническое колебание с постоянной составляющей частоты, которая задана заранее (начальная частота ГУН) и переменной составляющей, которая зависит от управляющего сигнала. Было показано, что мгновенная частота ГУН напрямую зависит от упраляющего сигнала (чем больше значение сигнала, тем больше частота ГУН)

Далее была собрана система ФАПЧ. Было проведено моделирвание с целью получить значение полосы захвата и полосы удержания. При заданных для моделирования параметров полоса захвата была равно 350 Гц. Полосу удержания промоделировать не удалось. Вместо этого была промоделирована ситуация с изменением частоты входного сигнала после синхронизации ФАПЧ.
Также было проведено моделирование, которое показало, что увеличение резонансной частоты (от которой напрямую зависят коэффициенты петлевого фильтра) позволяет быстрее захватывать сигнал, но с увеличением средней ошибки в процессе удержания.

Далее был реализован синтезатор частот на основе разработанного ФАПЧ. Моделирование показало, что полоса рабичих частот очень маленькая и необходимо постоянно подстраивать значение резонансной частоты для корректных результатов при различных коэффициентах деления R и N. Это доказывает, что представленная реализация сильно неидеальная и требует доработок
